# KUSTOMER FETCH DATA

In [ ]:
!pip install pytz

In [ ]:
C:\Users\maurice\Documents\Chat-Analyzer-V2\Chat Folder from Drive\2023-12-23-20231223T043422Z-001

In [ ]:
import requests
import pandas as pd
from datetime import datetime, date, timedelta
import pytz
import os

# Constants
KUSTOMER_API_URL = "https://edoofa.api.kustomerapp.com/v1/customers"
AUTH_TOKEN = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6IjY1MGVjMDVkNjg4MGQwNmJlNTNjZWVhMyIsInVzZXIiOiI2NTBlYzA1YmEzODAxZTQ4YjAwOGNhMDQiLCJvcmciOiI2M2JiZGEzNzY4ODFhYTZlMzdmZGRjNmMiLCJvcmdOYW1lIjoiZWRvb2ZhIiwidXNlclR5cGUiOiJtYWNoaW5lIiwicG9kIjoicHJvZDIiLCJyb2xlcyI6WyJvcmcucGVybWlzc2lvbi5tZXNzYWdlLnJlYWQiLCJvcmcucGVybWlzc2lvbi5ub3RlLnJlYWQiLCJvcmcudXNlci5jdXN0b21lci5yZWFkIiwib3JnLnVzZXIubWVzc2FnZS5yZWFkIiwib3JnLnVzZXIubm90ZS5yZWFkIiwib3JnLnBlcm1pc3Npb24uY3VzdG9tZXIucmVhZCJdLCJhdWQiOiJ1cm46Y29uc3VtZXIiLCJpc3MiOiJ1cm46YXBpIiwic3ViIjoiNjUwZWMwNWJhMzgwMWU0OGIwMDhjYTA0In0.IJI5P-BtBDCda9faVA3gfUYHA_rOZnWYuGM0np2Fbng"  # Replace with your actual API token
HEADERS = {
    "Authorization": f"Bearer {AUTH_TOKEN}",
    "Content-Type": "application/json"
}

mentor_mapping = {
    "64e5b276a184b56000839253": "Sharda Nayak",
    "63c01cb8bb84f3f762184233": "Saloni Rastogi",
    "645cc7e93d165050197cc502": "Ananya",
    "63c01c92a204a608c19c95cb": "Tenzin",
    "640e1696673d90d1182aadc6": "Jasmine Kaur",
    "63beab6b88e9f12575db3466": "Aditi Kapoor"
}

def fetch_customer_messages(customer_id):
    message_endpoint = f"{KUSTOMER_API_URL}/{customer_id}/messages"
    page = 1
    all_messages = []
    while True:
        response = requests.get(f"{message_endpoint}?page={page}&pageSize=1000", headers=HEADERS)
        if response.status_code != 200:
            break
        message_data = response.json()
        all_messages.extend(message_data['data'])
        if len(message_data['data']) < 1000:
            break
        page += 1
    return all_messages

def analyze_chats(chat_df, analysis_date):
    analysis_data = []
    unique_students = chat_df['Student'].unique()
    
    for student in unique_students:
        student_chats = chat_df[(chat_df['Student'] == student) & (chat_df['Date'].dt.date == analysis_date)]
        
        if student_chats.empty:
            missed_reply_mentor = 'No'
            missed_reply_student = 'No'
            broken_chat_mentor = 'No'
            broken_chat_student = 'No'
        else:
            # Check if mentor replied
            mentor_replied = not student_chats[student_chats['Direction'] == 'out'].empty
            # Check if student replied
            student_replied = not student_chats[student_chats['Direction'] == 'in'].empty

            # Determine missed replies
            missed_reply_mentor = 'Yes' if student_replied and not mentor_replied else 'No'
            missed_reply_student = 'Yes' if mentor_replied and not student_replied else 'No'

            # Determine broken chats
            last_message = student_chats.iloc[-1]
            broken_chat_mentor = 'Yes' if last_message['Direction'] == 'in' and mentor_replied else 'No'
            broken_chat_student = 'Yes' if last_message['Direction'] == 'out' and student_replied else 'No'

        analysis_data.append([student, missed_reply_mentor, missed_reply_student, broken_chat_mentor, broken_chat_student])

    return pd.DataFrame(analysis_data, columns=['Student', 'Missed Reply by Mentor', 'Missed Reply by Student', 'Broken Chat by Mentor', 'Broken Chat by Student'])

def main():
    # Set the date range for filtering
    # Define the current date in the 'Asia/Kolkata' timezone
    current_date = datetime.now(pytz.timezone("Asia/Kolkata"))

    # Define start date as the start of the day three days ago
    start_date = current_date - timedelta(days=1)
    start_date = start_date.replace(hour=0, minute=0, second=0, microsecond=0)

    # Define end date as the end of the day yesterday
    end_date = current_date  #timedelta(days=0)
    end_date = end_date.replace(hour=0, minute=0, second=0, microsecond=0)

    # Define the analysis date as two days ago
    analysis_date = current_date.date() - timedelta(days=1)

    response = requests.get(KUSTOMER_API_URL, headers=HEADERS)
    if response.status_code != 200:
        print("Failed to fetch customer data")
        return

    customers = response.json()['data']
    all_rows = []

    for customer in customers:
        customer_id = customer['id']
        first_name = customer['attributes'].get('firstName', 'Unknown')
        last_name = customer['attributes'].get('lastName', 'Unknown')
        full_name = f"{first_name} {last_name}"

        messages = fetch_customer_messages(customer_id)
        for message in messages:
            sent_at_utc = datetime.strptime(message['attributes']['sentAt'], '%Y-%m-%dT%H:%M:%S.%fZ').replace(tzinfo=pytz.utc)
            sent_at_ist = sent_at_utc.astimezone(pytz.timezone("Asia/Kolkata"))

            # Filter messages based on the date range
            if start_date <= sent_at_ist <= end_date:
                sent_at_local = sent_at_ist.strftime("%m/%d/%Y %H:%M:%S")
                direction = message['attributes']['direction']
                preview = message['attributes']['preview']
                user_id = message['relationships'].get('createdBy', {}).get('data', {}).get('id', None)
                mentor_name = mentor_mapping.get(user_id, "Unknown Mentor")

                all_rows.append([sent_at_local, preview, mentor_name, full_name, direction])

    chat_df = pd.DataFrame(all_rows, columns=['Date', 'Body', 'Mentor', 'Student', 'Direction'])
    chat_df['Date'] = pd.to_datetime(chat_df['Date'])
    analysis_df = analyze_chats(chat_df, analysis_date)

    # Save as Excel with multiple sheets
    try:
        base_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        base_dir = os.getcwd()
    excel_file_path = os.path.join(base_dir, 'kustomer_analysis15.xlsx')
    with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
        chat_df.to_excel(writer, sheet_name='Chat Data', index=False)
        analysis_df.to_excel(writer, sheet_name='Analysis', index=False)

    print(f"Analysis saved to {excel_file_path}")

if __name__ == "__main__":
    main()

# Table 2

In [1]:
import os
import pandas as pd
import re
from datetime import datetime, timedelta
import logging
from datetime import timedelta

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
# Set of processed chats to ensure each chat is analyzed only once
processed_chats = set()

# Function to process a single line of chat
def process_line(line):
    pattern = r'(\d{2}/\d{2}/\d{2}), (\d{1,2}:\d{2}\s?[APMapm]{2}) - (.*?): (.*)'
    match = re.match(pattern, line)
    if match:
        date_str, time_str, sender, message = match.groups()
        date = datetime.strptime(date_str, '%d/%m/%y')
        time = datetime.strptime(time_str, '%I:%M %p').strftime('%H:%M')
        return {'date': date, 'time': time, 'sender': sender, 'message': message}
    else:
        return None

def process_line_for_delay(line):
    pattern = r'(\d{2}/\d{2}/\d{2}), (\d{1,2}:\d{2}\s?[APMapm]{2}) - (.*?): (.*)'
    match = re.match(pattern, line)
    if match:
        date_str, time_str, sender, message = match.groups()
        datetime_str = date_str + ' ' + time_str
        datetime_obj = datetime.strptime(datetime_str, '%d/%m/%y %I:%M %p')
        return {'datetime': datetime_obj, 'sender': sender, 'message': message}
    else:
        return None

# Function to read the chat file specifically for delay calculation
def read_chat_file_for_delay(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    concatenated_lines = []
    current_message = ""
    date_pattern = re.compile(r'^\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2}\s?[APMapm]{2} - ')

    for line in lines:
        if date_pattern.match(line):
            if current_message:
                concatenated_lines.append(current_message)
            current_message = line.rstrip()
        else:
            # Append this line to the current message, if it's not empty
            if current_message:
                current_message += " " + line.strip()
    if current_message:
        concatenated_lines.append(current_message)

    return pd.DataFrame([process_line_for_delay(line) for line in concatenated_lines if process_line_for_delay(line) is not None])

# The process_line_for_delay function remains the same
       
    
def calculate_time_spent_student(chat_df, target_date, employee_name):
    student_messages = chat_df[(chat_df['date'].dt.date == target_date) & (chat_df['sender'] != employee_name)]
    total_chars = student_messages['message'].str.len().sum()
    time_spent_seconds = (total_chars / 10) * 5
    return strfdelta(timedelta(seconds=time_spent_seconds)) 

def format_delay_time(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    if hours > 0:
        return f"{hours} hr {minutes} mins"
    else:
        return f"{minutes} mins"

def calculate_employee_delay(chat_df, employee_name, target_date):
    delays = []
    delay_times = []
    messages_for_reference = []

    last_student_message_time = None
    employee_responded_after_student = False

    delay_counter = 1

    for index, row in chat_df.iterrows():
        # Ensure datetime is correctly formatted
        row_date = row['datetime'].date()

        if row_date != target_date:
            continue

        if row['sender'] != employee_name:
            last_student_message_time = row['datetime']
            employee_responded_after_student = False
        elif row['sender'] == employee_name and last_student_message_time is not None and not employee_responded_after_student:
            time_diff = row['datetime'] - last_student_message_time

            if time_diff > timedelta(minutes=15):
                formatted_time = row['datetime'].strftime('%d %b %Y %I:%M%p')
                delays.append(f"{delay_counter} - {formatted_time}")
                delay_counter += 1

                delay_times.append(format_delay_time(time_diff.seconds))

                start_index = max(0, index - 5)
                end_index = min(index + 5, len(chat_df))
                context_messages = chat_df.iloc[start_index:end_index]

                formatted_context_messages = [f"{idx+1} - {message_row['datetime'].strftime('%d %b %Y %I:%M%p')} - {message_row['sender']}: {message_row['message']}" for idx, message_row in context_messages.iterrows()]
                formatted_message = "\n".join(formatted_context_messages)
                messages_for_reference.append(formatted_message)

            employee_responded_after_student = True

    # Combine the lists into multiline strings
    delays_str = "\n".join(delays)
    delay_times_str = "\n".join(delay_times)
    messages_for_reference_str = "\n".join(messages_for_reference)

    return delays_str, delay_times_str, messages_for_reference_str

def is_broken_chat_student(chat_df, employee_name, target_date):
    # Adjust for the previous day
    previous_day = target_date - timedelta(days=1)

    # Filter messages for the previous day
    daily_messages = chat_df[chat_df['date'].dt.date == previous_day]

    # Check if both the employee and the student had at least one text on that day
    if daily_messages.empty:
        return 'No'  # No conversation happened on this day

    employee_messages = daily_messages[daily_messages['sender'] == employee_name]
    student_messages = daily_messages[daily_messages['sender'] != employee_name]

    # Check if both parties have sent at least one message
    if employee_messages.empty or student_messages.empty:
        return 'No'  # One of the parties did not send any message

    # Check the sender of the last message of the day
    last_message_sender = daily_messages.iloc[-1]['sender']

    # If the last message is from the employee, it's a broken chat by the student
    if last_message_sender == employee_name:
        return 'Yes'

    return 'No'
    
def count_missed_replies_student(chat_df, employee_name, report_date):
    count = 0
    for i in range(7):
        check_date = report_date - timedelta(days=i)
        messages_on_date = chat_df[chat_df['date'].dt.date == check_date]
        employee_messages = messages_on_date[messages_on_date['sender'] == employee_name]
        student_messages = messages_on_date[messages_on_date['sender'] != employee_name]
        if employee_messages.empty and not student_messages.empty:
            count += 1
    return count

def count_missed_replies_studentt(chat_df, employee_name, target_date):
    previous_day = target_date - timedelta(days=1)
    #print(f"Checking for missed replies on: {previous_day}")

    daily_messages = chat_df[chat_df['date'].dt.date == previous_day]
    
    employee_messages = daily_messages[daily_messages['sender'] == employee_name]
    student_messages = daily_messages[daily_messages['sender'] != employee_name]

    #print(f"Employee messages count: {len(employee_messages)}")
    #print(f"Student messages count: {len(student_messages)}")

    # Check if there are messages from the employee but none from the student
    if not employee_messages.empty and student_messages.empty:
        #print("Missed reply found.")
        return 'Yes'
    
    #print("No missed reply.")
    return 'No'

def count_missed_replies_last_7_days(chat_df, employee_name, start_date):
    missed_replies_count = 0

    # Iterate over the last 7 days
    for i in range(7):
        date_to_check = start_date - timedelta(days=i)
        messages_on_date = chat_df[chat_df['date'].dt.date == date_to_check]

        # Check if there are messages on this date
        if not messages_on_date.empty:
            # Check if there's any message from the employee on this day
            employee_messages = messages_on_date[messages_on_date['sender'] == employee_name]
            
            # Check if there's any message from others (leads) on this day
            lead_messages = messages_on_date[messages_on_date['sender'] != employee_name]

            # If there are messages from others but none from the employee, count as a missed reply
            if not lead_messages.empty and employee_messages.empty:
                missed_replies_count += 1

    return missed_replies_count

# Function to calculate the time spent in chat based on message lengths
def calculate_time_spent(chat_df, target_date, employee_name, team_folder):
    daily_messages = chat_df[(chat_df['date'].dt.date == target_date) & (chat_df['sender'] == employee_name)]

    if daily_messages.empty:
        return '00:00:00'

    total_time_spent_seconds = 0

    for index, row in daily_messages.iterrows():
        message = row['message']
        total_chars = len(message)

        if team_folder == "KAM" and total_chars > 700:
            time_spent_seconds = 5  # Cap at 5 seconds for long messages in KAM team folder
        else:
            time_spent_seconds = total_chars * 0.2  # Assuming 0.2 seconds per character

        total_time_spent_seconds += time_spent_seconds

        # Optional: Debug print
        #print(f"New Chat Found in {team_folder}")
        #print(f"Message Length: '{total_chars}'")
        #print(f"Message: '{message}' - Time Spent: {time_spent_seconds} seconds")

    time_spent_timedelta = timedelta(seconds=total_time_spent_seconds)
    formatted_time_spent = strfdelta(time_spent_timedelta)

    return formatted_time_spent

def strfdelta(tdelta):
    hours, remainder = divmod(tdelta.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"   

def calculate_lead_response(chat_df, employee_name, target_date):
    """
    Calculates the lead response based on the number of texts exchanged between the employee and others (non-employee).
    - If only the employee sent messages, the response is 100%.
    - If only non-employees sent messages, the response is 0%.
    - Otherwise, it's the ratio of the employee's texts to the total texts, constrained between 1% and 99%.
    """
    # Filter messages for the target date
    daily_messages = chat_df[chat_df['date'].dt.date == target_date]
    
    # Check if there are any messages from the employee and others on the target date
    employee_messages = daily_messages[daily_messages['sender'] == employee_name]
    non_employee_messages = daily_messages[daily_messages['sender'] != employee_name]

    # If only the employee sent messages
    if not employee_messages.empty and non_employee_messages.empty:
        return 0

    # If only non-employees sent messages
    if employee_messages.empty and not non_employee_messages.empty:
        return 100

    # If both the employee and non-employees have sent messages
    if not employee_messages.empty and not non_employee_messages.empty:
        num_texts_employee = len(employee_messages)
        num_texts_non_employee = len(non_employee_messages)
        total_texts = num_texts_employee + num_texts_non_employee
        lead_response = (num_texts_employee / total_texts) * 100
        return max(min(lead_response, 99), 1)

    # If neither the employee nor non-employees sent messages
    return 0



def calculate_broken_chat_within_working_hrs(chat_df, employee_name, report_date, team_folder):
    # Adjust working hours based on team folder
    if team_folder == 'EWYL':
        # KAM Team working hours: 7:30 AM to 4:00 PM
        work_start_time = datetime(report_date.year, report_date.month, report_date.day, 7, 30, 0)
        work_end_time = datetime(report_date.year, report_date.month, report_date.day, 16, 30, 0)
    else:
        # Default working hours for other teams: 12:00 AM to 12:00 PM
        work_start_time = datetime(report_date.year, report_date.month, report_date.day, 0, 0, 0)
        work_end_time = datetime(report_date.year, report_date.month, report_date.day, 23, 59, 0)

    # Filter the messages for the given report date
    messages_on_date = chat_df[chat_df['datetime'].dt.date == report_date.date()]

    # Check if both employee and lead have messaged on the report date
    if not messages_on_date.empty:
        employee_messages = messages_on_date[messages_on_date['sender'] == employee_name]
        lead_messages = messages_on_date[messages_on_date['sender'] != employee_name]
        last_message_sender = messages_on_date.iloc[-1]['sender']
        # Ensure both the employee and lead have at least one message
        if not employee_messages.empty and not lead_messages.empty:
            # Get the time of the last message from the lead
            last_lead_message_time = lead_messages.iloc[-1]['datetime']

            # Check if the last message is from the lead and within working hours
            if last_message_sender != employee_name and (work_start_time <= last_lead_message_time <= work_end_time):
                return 'Yes'  # Broken chat if last message from the lead is within working hours
            else:
                return 'No'  # Not a broken chat if last message from the lead is outside working hours or from the employee

    # If no messages or only one party messaged, return 'No'
    return 'No'

def calculate_broken_chat(chat_df, employee_name, report_date):
    # Filter the messages for the given report date
    messages_on_date = chat_df[chat_df['date'].dt.date == report_date]
    
    # Check if both employee and lead have messaged on the report date
    if not messages_on_date.empty:
        employee_messages = messages_on_date[messages_on_date['sender'] == employee_name]
        lead_messages = messages_on_date[messages_on_date['sender'] != employee_name]
        
        # Ensure both the employee and lead have at least one message
        if not employee_messages.empty and not lead_messages.empty:
            # Check if the last message is from the lead
            last_message_sender = messages_on_date.iloc[-1]['sender']
            return last_message_sender != employee_name

    

def missed_replies_employee(chat_df, employee_name, target_date):
    # Filter messages for the target date
    daily_messages = chat_df[chat_df['date'].dt.date == target_date]
    
    # Check if there are any messages from the student
    student_messages = daily_messages[daily_messages['sender'] != employee_name]
    
    # Check if there are any messages from the employee
    employee_messages = daily_messages[daily_messages['sender'] == employee_name]
    
    # If there are student messages but no employee messages, consider it a missed reply
    if not student_messages.empty and employee_messages.empty:
        return 'Yes'
    
    return 'No'

def missed_replies_employee_after_working_hrs(chat_df, employee_name, target_date, team_folder):
    # Check if the team folder is 'KAM Team' to define different working hours
    if team_folder == 'EWYL':
        # EWYL Team working hours: 7:30 AM to 4:00 PM
        work_start_time = datetime(target_date.year, target_date.month, target_date.day, 7, 30, 0)
        work_end_time = datetime(target_date.year, target_date.month, target_date.day, 16, 30, 0)
    
    elif team_folder == 'KAM':
       # KAM Team working hours: 7:30 AM to 4:00 PM
        work_start_time = datetime(target_date.year, target_date.month, target_date.day, 11, 30, 0)
        work_end_time = datetime(target_date.year, target_date.month, target_date.day, 21, 30, 0)

    else:
        # Default working hours for other teams: 12:00 AM to 12:00 PM
        work_start_time = datetime(target_date.year, target_date.month, target_date.day, 0, 0, 0)
        work_end_time = datetime(target_date.year, target_date.month, target_date.day, 23, 59, 0)

    # Ensure target_date is a datetime object
    if not isinstance(target_date, datetime):
        raise ValueError("target_date must be a datetime object")

    # Filter messages for the target date using 'datetime' column
    daily_messages = chat_df[chat_df['datetime'].dt.date == target_date.date()]
    
    # Check if there are any messages from the student
    student_messages = daily_messages[daily_messages['sender'] != employee_name]
    
    # Check if there are any messages from the employee
    employee_messages = daily_messages[daily_messages['sender'] == employee_name]

    # If there are student messages but no employee messages
    if not student_messages.empty and employee_messages.empty:
        print(f"ENTER THE LOOP:{student_messages}")
        # Check if any student message is within working hours
        for _, row in student_messages.iterrows():
            message_time = row['datetime']
            if work_start_time <= message_time <= work_end_time:
                return 'Yes'  # Missed reply within working hours
        return 'No'  # All student messages are outside working hours

    return 'No'

# Function to read the chat file and return a dataframe
def read_chat_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    concatenated_lines = []
    current_message = ""
    date_pattern = re.compile(r'^\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2}\s?[APMapm]{2} - ')

    for line in lines:
        if date_pattern.match(line):
            if current_message:
                concatenated_lines.append(current_message)
            current_message = line.rstrip()
        else:
            # Append this line to the current message, if it's not empty
            if current_message:
                current_message += " " + line.strip()
    if current_message:
        concatenated_lines.append(current_message)

    return pd.DataFrame([process_line(line) for line in concatenated_lines if process_line(line) is not None])

# process_line function remains the same


# Function to extract the chat file name based on a specific pattern
def extract_chat_name(chat_file_name):
    #logging.debug(f"Extracting name from chat file: {chat_file_name}")
    match = re.search(r'WhatsApp Chat with _?(.*?)(?:\(\d+\))?_?\.txt', chat_file_name)
    if match:
        extracted_name = re.sub(r'_*(?:\(\d+\))?$', '', match.group(1))
        #logging.debug(f"Extracted chat name: {extracted_name}")
        return extracted_name
    #logging.warning(f"No match found for chat file name: {chat_file_name}")
    return None

# Function to calculate the total count of missed replies by the student for the last N days
def calculate_total_count_missed_replies_student(chat_df, employee_name, num_days):
    total_count = 0

    # Iterate over the last N days
    for i in range(num_days):
        target_date = report_date - timedelta(days=i)
        count = count_missed_replies_student(chat_df, employee_name, target_date)
        total_count += count

    return total_count

# Function to extract the start date from a chat file
def get_chat_start_date(chat_df):
    if not chat_df.empty:
        #logging.debug(f"Extracted chat name: {chat_df['date'].min().date()}")
        return chat_df['date'].min().date()
    return None

# Function to process a single chat file
def process_chat_file(file_path, report_date, main_directory, team_folder, employee_folder):
    global processed_chats
    broken_chat_count = 0
    missed_reply_count = 0
    max_time_spent = ('', '00:00:00')  # (date, time)

    chat_file_name = os.path.basename(file_path)
    #logging.debug(f"Processing chat file: {chat_file_name}")

    chat_name = extract_chat_name(chat_file_name)
    #logging.debug(f"Chat name extracted: {chat_name}")



    

    if chat_name is None:
        #logging.warning(f"Chat name could not be extracted for file: {chat_file_name}")
        return None
    if chat_name in processed_chats:
        #logging.info(f"Chat file already processed: {chat_name}")
        return None

    processed_chats.add(chat_name)
    #logging.debug(f"Chat file added to processed list: {chat_name}")

    chat_df = read_chat_file(file_path)
    if chat_df.empty:
        #logging.warning(f"Chat DataFrame is empty for file: {chat_file_name}")
        return None
    chat_start_date = get_chat_start_date(chat_df)

    # Use read_chat_file_for_delay specifically for delay calculations
    chat_df_for_delay = read_chat_file_for_delay(file_path)
    if chat_df_for_delay.empty:
        return None

    
    target_date = report_date - timedelta(days=1)

    # Call the function with the target date
    delays, delay_times, messages_for_reference = calculate_employee_delay(chat_df_for_delay, employee_folder, target_date)
    



    # Calculate the time spent on Day 0 (report_date - 1 day)
    day_0_date = report_date - timedelta(days=1)
    day_0_time_spent = calculate_time_spent(chat_df, day_0_date, employee_folder,team_folder)
    #day_0_time_spent = timedelta(hours=int(day_0_time_spent.split(':')[0]), minutes=int(day_0_time_spent.split(':')[1]), seconds=int(day_0_time_spent.split(':')[2]))
    # Calculate the lead response based on text length difference for Day 0
    lead_response_day_0 = calculate_lead_response(chat_df, employee_folder, day_0_date)
     
    day_0_date = report_date - timedelta(days=1)  # Assuming report_date is a datetime object of the target date
    broken_chat = calculate_broken_chat(chat_df, employee_folder, day_0_date)
    missed_replies_day_0 = missed_replies_employee(chat_df, employee_folder, day_0_date)

    # Get the current datetime
    today_datetime = datetime.now()

    # Get today's date with time part (assuming start of the day as default)
    today_date_1 = today_datetime.replace(hour=0, minute=0, second=0, microsecond=0)

    # If today_date.weekday() logic remains the same
    # ...

    # Now, when you calculate target_date, it remains a datetime object
    target_date_1 = today_date_1 - timedelta(days=1)


    Actual_missed_Reply = missed_replies_employee_after_working_hrs(chat_df_for_delay, employee_folder, target_date_1,team_folder)
    Actual_broken_chat = calculate_broken_chat_within_working_hrs(chat_df_for_delay, employee_folder, target_date_1,team_folder)
    total_count_missed_reply_student = calculate_total_count_missed_replies_student(chat_df, employee_folder, 14)
    


    
    for i in range(7):  # Assuming you're checking the past 7 days
        check_date = report_date - timedelta(days=i)
        
        # Check for broken chats
        if calculate_broken_chat(chat_df, employee_folder, check_date):
            broken_chat_count += 1

        # Check for missed replies
        start_date_for_missed_replies = report_date - timedelta(days=7)
        
        missed_replies_count = count_missed_replies_last_7_days(chat_df, employee_folder, start_date_for_missed_replies)

        # Check for max time spent
        time_spent = calculate_time_spent(chat_df, check_date, employee_folder,team_folder)
        if time_spent > max_time_spent[1]:
            max_time_spent = (check_date.strftime('%m-%d-%Y'), time_spent)
    
    
    
    # Calculate the time spent by students
    time_spent_student = {
    'Day 0 (Student)': calculate_time_spent_student(chat_df, report_date - timedelta(days=1), employee_folder),
    'Day 1 (Student)': calculate_time_spent_student(chat_df, report_date - timedelta(days=2), employee_folder),
    'Day 2 (Student)': calculate_time_spent_student(chat_df, report_date - timedelta(days=3), employee_folder),
    }
    
    # Inside the process_chat_file function, after calculating day_0_time_spent:
    day_0_date = report_date - timedelta(days=1)
    day_1_date = report_date - timedelta(days=2)
    day_2_date = report_date - timedelta(days=3)

    # Calculate time spent by student for Day 0, Day 1, and Day 2
    time_spent_student_day_0 = calculate_time_spent_student(chat_df, day_0_date, employee_folder)
    time_spent_student_day_1 = calculate_time_spent_student(chat_df, day_1_date, employee_folder)
    time_spent_student_day_2 = calculate_time_spent_student(chat_df, day_2_date, employee_folder)



    total_time_spent_student = time_spent_student['Day 0 (Student)']

    broken_chat_student = is_broken_chat_student(chat_df, employee_folder, report_date)

    mmissed_reply_student = count_missed_replies_studentt(chat_df, employee_folder, report_date)

    # Calculate the time spent on each day relative to the report date
    time_spent = {}
    for i in range(1, 4):  # Start from 1 since we want Day 0 to be the day before the report date
        target_date = report_date - timedelta(days=i)
        time_spent[f'Day {i-1}'] = calculate_time_spent(chat_df, target_date, employee_folder,team_folder)
    
    target_date_for_chat_start_Date = report_date - timedelta(days=1)
    day_indicator_employee = (target_date_for_chat_start_Date - chat_start_date).days
    #logging.debug(f"day indicator employee: {day_indicator_employee}")
    # Determine Day Indicator based on chat start date
    if chat_start_date:
        if (target_date_for_chat_start_Date - chat_start_date).days == 0:
          
            day_indicator = 'Day 0'
        elif (target_date_for_chat_start_Date - chat_start_date).days == 1:
            
            day_indicator = 'Day 1'
        elif (target_date_for_chat_start_Date - chat_start_date).days == 2:
            
            day_indicator = 'Day 2'

        elif (target_date_for_chat_start_Date - chat_start_date).days == 3:
            
            day_indicator = 'OLD'

        else:
            day_indicator = 'OLD'  # For chats that don't fall into Day 0, Day 1, or Day 2

    

    main_directory_name = os.path.basename(main_directory)

    row = {
        'Date': main_directory_name,
        'Team Folder': team_folder,
        'Employee Folder': employee_folder,
        'Chat File Name': chat_name,        # Inserting the extracted chat name
        'Day Indicator': day_indicator,
        'Total_Time_Spent_Employee': day_0_time_spent,  # Use Day 0 time as the total time spent
        'LR': lead_response_day_0,  # Add the lead response here
        'Broken Chat(Employee)': 'Yes' if broken_chat else 'No',
        'Missed Replies (Employee)': missed_replies_day_0,
        'Count_Broken_Chat_Employee (Employee)': broken_chat_count,
        'Count_Missed_Replies_Missed_Replies (Employee)': missed_replies_count,
        'Date of Max time': max_time_spent[0],
        **time_spent,
        'Total Time Spent (Student)': total_time_spent_student,
        'Broken Chat (Student)': broken_chat_student,
        'Missed Replies (Student)': mmissed_reply_student,
        'Day_0_Time_Spent_Student': time_spent_student_day_0,  # Add time spent for Day 0
        'Day_1_Time_Spent_Student': time_spent_student_day_1,  # Add time spent for Day 1
        'Day_2_Time_Spent_Student': time_spent_student_day_2,
        'Total_Count_Missed_Reply_Chat_Student' : total_count_missed_reply_student,
        'Employee Delays': delays,
        'Delay Durations': delay_times,
        'Messages for Reference': messages_for_reference,
        'Actual Missed Reply from Employee': Actual_missed_Reply,
        'Actual Broken Chat from Employee' : Actual_broken_chat,
    }
    #logging.debug(f"Row created for chat: {row}")
    return row

# Main directory path construction and report date setting
# Main directory path construction and report date setting
local_date_format = '%Y-%m-%d'  # Adjust this to your local date format
main_directory = 'C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\2023-12-23-20231223T043422Z-001'  # Replace with your base directory path

# Get today's date
today_date = datetime.now().date()

# Check the day of the week
if today_date.weekday() in range(1, 6):  # Tuesday to Saturday
    report_date = today_date
else:  # Monday
    # You can adjust the number of days to subtract based on your specific requirements
    report_date = today_date - timedelta(days=1)
    logging.debug(f"Report Date: {report_date}")
report_date_str_local = report_date.strftime(local_date_format)
main_directory_path = os.path.join(main_directory, report_date_str_local)

# Function to navigate through the directory structure and process all chat files
def process_team_folders(main_directory, report_date):
    all_data = []
    for team_folder in os.listdir(main_directory):
        team_path = os.path.join(main_directory, team_folder)
        if not os.path.isdir(team_path):
            continue
        for employee_folder in os.listdir(team_path):
            employee_path = os.path.join(team_path, employee_folder)
            if not os.path.isdir(employee_path):
                continue
            for chat_file in os.listdir(employee_path):
                chat_file_path = os.path.join(employee_path, chat_file)
                row = process_chat_file(chat_file_path, report_date, main_directory, team_folder, employee_folder)
                if row:
                    all_data.append(row)
                    logging.debug(f"Row appended for chat file: {chat_file}")
    df = pd.DataFrame(all_data)
    #logging.debug(f"DataFrame constructed with {len(df)} rows")
    return df

# Process all team folders and chats based on the report date
all_chats_df = process_team_folders(main_directory_path, report_date)


# Save to a CSV file
csv_file_path = 'C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\2023-12-23-20231223T043422Z-001\\chat_data6.csv'  # Define your desired path and file name
all_chats_df.to_csv(csv_file_path, index=False)
#print(f"DataFrame saved as CSV at {csv_file_path}")
# Display the result
#print(all_chats_df)


2023-12-23 11:04:43,616 - DEBUG - Row appended for chat file: WhatsApp Chat with Abraham EWYL22E0848.txt
2023-12-23 11:04:43,880 - DEBUG - Row appended for chat file: WhatsApp Chat with Alson EWYL22E0675.txt
2023-12-23 11:04:44,111 - DEBUG - Row appended for chat file: WhatsApp Chat with Anesu EWYL23E0893.txt
2023-12-23 11:04:44,331 - DEBUG - Row appended for chat file: WhatsApp Chat with Ashley EWYL22E0726.txt
2023-12-23 11:04:44,596 - DEBUG - Row appended for chat file: WhatsApp Chat with Ashley EWYL22E1272.txt


ENTER THE LOOP:               datetime           sender  \
270 2023-12-22 07:23:00  +27 84 355 4583   

                                               message  
270  ```FOREX GURU INVESTMENT MINING !!! The bigges...  


2023-12-23 11:04:44,861 - DEBUG - Row appended for chat file: WhatsApp Chat with Ayanda EWYL23E0703.txt
2023-12-23 11:04:45,129 - DEBUG - Row appended for chat file: WhatsApp Chat with Belinda EWYL23E0059.txt
2023-12-23 11:04:45,380 - DEBUG - Row appended for chat file: WhatsApp Chat with Bethel EWYL22E0867.txt
2023-12-23 11:04:45,602 - DEBUG - Row appended for chat file: WhatsApp Chat with Blessing EWYL23E0106.txt
2023-12-23 11:04:45,783 - DEBUG - Row appended for chat file: WhatsApp Chat with BOTHWELL EWYL23E0119.txt
2023-12-23 11:04:46,054 - DEBUG - Row appended for chat file: WhatsApp Chat with Caroline EWYL23E0276.txt
2023-12-23 11:04:46,342 - DEBUG - Row appended for chat file: WhatsApp Chat with Cecilia EWYL23E0615.txt
2023-12-23 11:04:46,628 - DEBUG - Row appended for chat file: WhatsApp Chat with Charlene EWYL23E0608.txt


ENTER THE LOOP:               datetime            sender  \
375 2023-12-22 12:39:00  +263 78 517 8501   
376 2023-12-22 12:40:00  +263 78 517 8501   
377 2023-12-22 12:41:00  +263 78 517 8501   
378 2023-12-22 12:43:00  +263 78 517 8501   

                                               message  
375  *Application form to be shared *  *Offline App...  
376  *Application form to be shared *  *Offline App...  
377  *Application form to be shared *  *Offline App...  
378  *Application form to be shared *  *Offline App...  


2023-12-23 11:04:46,895 - DEBUG - Row appended for chat file: WhatsApp Chat with Chidochashe EWYL23E0778.txt
2023-12-23 11:04:47,178 - DEBUG - Row appended for chat file: WhatsApp Chat with Chiedza EWYL23E0865.txt
2023-12-23 11:04:47,397 - DEBUG - Row appended for chat file: WhatsApp Chat with Delight EWYL22E0377.txt
2023-12-23 11:04:47,676 - DEBUG - Row appended for chat file: WhatsApp Chat with Donavan EWYL23E0878.txt
2023-12-23 11:04:47,896 - DEBUG - Row appended for chat file: WhatsApp Chat with Eleanor EWYL22E0996.txt
2023-12-23 11:04:48,163 - DEBUG - Row appended for chat file: WhatsApp Chat with Elizabeth EWYL23E1000.txt
2023-12-23 11:04:48,412 - DEBUG - Row appended for chat file: WhatsApp Chat with Ellary EWYL23E0906.txt
2023-12-23 11:04:48,617 - DEBUG - Row appended for chat file: WhatsApp Chat with Getrude EWYL23E1062.txt
2023-12-23 11:04:48,925 - DEBUG - Row appended for chat file: WhatsApp Chat with Hove EWYL23E0076.txt
2023-12-23 11:04:49,225 - DEBUG - Row appended for ch

...

2023-12-23 11:07:33,444 - DEBUG - Row appended for chat file: WhatsApp Chat with SIMPOSO MWAANGWA_EDOOFAZAMBNGS(12_12)GS.txt
2023-12-23 11:07:33,700 - DEBUG - Row appended for chat file: WhatsApp Chat with Sizalobuhle Ndebele_EDOOFA(20_12)GS.txt
2023-12-23 11:07:33,995 - DEBUG - Row appended for chat file: WhatsApp Chat with Tadiwa Mukoriwo_EDOOFA(12_12)GS.txt
2023-12-23 11:07:34,342 - DEBUG - Row appended for chat file: WhatsApp Chat with Tadiwanashe Matina_EDOOFA(14_12)GS.txt
2023-12-23 11:07:34,677 - DEBUG - Row appended for chat file: WhatsApp Chat with Tadiwanashe Reginald Veva_EDOOFA(12_12)GS.txt
2023-12-23 11:07:34,915 - DEBUG - Row appended for chat file: WhatsApp Chat with Tafadzwa Kainama_EDOOFA(19_12)GS.txt
2023-12-23 11:07:35,196 - DEBUG - Row appended for chat file: WhatsApp Chat with Tafadzwa Makumire_EDOOFA(14_12)GS.txt
2023-12-23 11:07:35,433 - DEBUG - Row appended for chat file: WhatsApp Chat with Tafadzwa Njanji_EDOOFA(19_12)GS.txt
2023-12-23 11:07:35,699 - DEBUG - Ro

2023-12-23 11:07:53,522 - DEBUG - Row appended for chat file: WhatsApp Chat with Alicia_Edoofa(16_10)PG.txt
2023-12-23 11:07:53,812 - DEBUG - Row appended for chat file: WhatsApp Chat with Andile_Edoofa(18_12)PG-SH.txt
2023-12-23 11:07:54,158 - DEBUG - Row appended for chat file: WhatsApp Chat with Anotida_Edoofa(22_12)PG.txt
2023-12-23 11:07:54,635 - DEBUG - Row appended for chat file: WhatsApp Chat with Arnold_Edoofa(14_12)PG.txt
2023-12-23 11:07:55,023 - DEBUG - Row appended for chat file: WhatsApp Chat with Asmir_Edoofa(22_12)PG.txt
2023-12-23 11:07:55,313 - DEBUG - Row appended for chat file: WhatsApp Chat with Billy_Edoofa(19_12)PG-sl.txt
2023-12-23 11:07:55,612 - DEBUG - Row appended for chat file: WhatsApp Chat with Brenda_Edoofa(15_12)PG-IE.txt
2023-12-23 11:07:56,005 - DEBUG - Row appended for chat file: WhatsApp Chat with Catherine_Edoofa(19_12)PG-SL.txt
2023-12-23 11:07:56,487 - DEBUG - Row appended for chat file: WhatsApp Chat with Charmaine_Edoofa(20_12)PG-IE.txt
2023-12-

2023-12-23 11:08:20,707 - DEBUG - Row appended for chat file: WhatsApp Chat with Isheanesu_Edoofa(11_12)HP.txt
2023-12-23 11:08:21,105 - DEBUG - Row appended for chat file: WhatsApp Chat with Ivainashe_Edoofa(30_10)HP.txt
2023-12-23 11:08:21,507 - DEBUG - Row appended for chat file: WhatsApp Chat with Joshua_Edoofa(20_11)HP.txt
2023-12-23 11:08:21,890 - DEBUG - Row appended for chat file: WhatsApp Chat with Juliet Malowa Edoofa (27_11) SZ 2025.txt
2023-12-23 11:08:22,290 - DEBUG - Row appended for chat file: WhatsApp Chat with Kabongwe K Sibanda Edoofa (11_10) SZ.txt
2023-12-23 11:08:22,690 - DEBUG - Row appended for chat file: WhatsApp Chat with Kayembe Israel Edoofa(30_10)SR300.txt
2023-12-23 11:08:23,042 - DEBUG - Row appended for chat file: WhatsApp Chat with Kudangirana Rufaro Edoofa (12_12) SZ.txt
2023-12-23 11:08:23,394 - DEBUG - Row appended for chat file: WhatsApp Chat with Kunashe Joel Munyoro Edoofa (30_11) SZ 2025.txt
2023-12-23 11:08:23,773 - DEBUG - Row appended for chat 

2023-12-23 11:08:46,670 - DEBUG - Row appended for chat file: WhatsApp Chat with Chats Export (KAM Team).txt
2023-12-23 11:08:46,995 - DEBUG - Row appended for chat file: WhatsApp Chat with Chirimhanzi Tinodaishe Edoofa (05_12) SZ 2025.txt
2023-12-23 11:08:47,314 - DEBUG - Row appended for chat file: WhatsApp Chat with Chiturumani Takudzwa Edoofa (05_12) SZ.txt
2023-12-23 11:08:47,645 - DEBUG - Row appended for chat file: WhatsApp Chat with Joshua Tatenda Chipindu Edoofa (22_11) SZ 2025.txt
2023-12-23 11:08:47,946 - DEBUG - Row appended for chat file: WhatsApp Chat with Mnqobi Bhebhe Edoofa (13_12) SZ.txt
2023-12-23 11:08:48,251 - DEBUG - Row appended for chat file: WhatsApp Chat with Njanji Ruvarashe S Edoofa (28_11) SZ 2025.txt
2023-12-23 11:08:48,562 - DEBUG - Row appended for chat file: WhatsApp Chat with Phiri Simphiwe Edoofa (08_12) SZ.txt
2023-12-23 11:08:48,863 - DEBUG - Row appended for chat file: WhatsApp Chat with Rumbidzai Susan Joseph Edoofa (30_11) SZ.txt
2023-12-23 11:08

2023-12-23 11:09:09,002 - DEBUG - Row appended for chat file: WhatsApp Chat with Hazel MamombeIE_EA(21_12)TR.txt
2023-12-23 11:09:09,219 - DEBUG - Row appended for chat file: WhatsApp Chat with Huldah Nkomo_EA(20_12)TR.txt
2023-12-23 11:09:09,451 - DEBUG - Row appended for chat file: WhatsApp Chat with Kudiwa Takafakare_EA(19_12)TR.txt
2023-12-23 11:09:09,655 - DEBUG - Row appended for chat file: WhatsApp Chat with Likezo Mufuzi_EDOOFAZAMBNGS(19_12)TR.txt
2023-12-23 11:09:09,946 - DEBUG - Row appended for chat file: WhatsApp Chat with Luckson Kapishe_EA(20_12)TR.txt
2023-12-23 11:09:10,216 - DEBUG - Row appended for chat file: WhatsApp Chat with Luyando Siasulingana_EA(15_12)TR.txt
2023-12-23 11:09:10,467 - DEBUG - Row appended for chat file: WhatsApp Chat with Madyedzo Everton_EA(12_12)TR.txt
2023-12-23 11:09:10,670 - DEBUG - Row appended for chat file: WhatsApp Chat with Makuvaza Blessed_EA(20_12)TR.txt
2023-12-23 11:09:10,947 - DEBUG - Row appended for chat file: WhatsApp Chat with 

In [ ]:
Table 4

In [ ]:
import os
import pandas as pd
import datetime
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Function to list all chat files in the directory structure
def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files

def parse_chat_file(file_path, expected_date_minus_one, person_name):
    chat_data = []
    last_non_person_time = None  # Tracks the time of the last non-person message

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
            system_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*)', line)
            
            if message_match:
                date_time_str, _, message = message_match.groups()
                sender = person_name  # Use person_name as sender
            elif system_match:
                date_time_str, info = system_match.groups()
                sender = None
            else:
                continue

            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

            if date_time.date() != expected_date_minus_one:
                continue

            message_type = 'person' if sender == person_name else 'other'

            # Calculate delay
            delay = False
            if message_type == 'person' and last_non_person_time:
                diff = date_time - last_non_person_time
                delay = diff.total_seconds() > 900  # 15 minutes in seconds

            chat_data.append((date_time, sender, message_type, delay))

            if message_type == 'other':
                last_non_person_time = date_time

    return chat_data


def create_template_dataframe():
    times = [datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(minutes=1 * i) for i in range(1440)]
    intervals = [time.strftime('%I:%M %p') for time in times]
    df = pd.DataFrame(index=intervals)
    return df

def populate_dataframe(df, parsed_data, start_column_index):
    new_columns = {}

    for entry in parsed_data:
        date_time, sender, message_type, delay = entry
        interval_index = min((date_time.hour * 60 + date_time.minute) // 1, 1439)
        interval = df.index[interval_index]

        if start_column_index not in new_columns:
            new_columns[start_column_index] = pd.Series(0, index=df.index)  # For 'person'
        if start_column_index + 1 not in new_columns:
            new_columns[start_column_index + 1] = pd.Series(0, index=df.index)  # For 'other'
        if start_column_index + 2 not in new_columns:
            new_columns[start_column_index + 2] = pd.Series(False, index=df.index)  # For delay column

        if message_type == 'person':
            new_columns[start_column_index].at[interval] = 1
        elif message_type == 'other':
            new_columns[start_column_index + 1].at[interval] = 1

        new_columns[start_column_index + 2].at[interval] = delay  # Set delay flag

    df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)

    return df, start_column_index + 3


def process_person_chats(chat_files):
    dataframes = {}
    for file in chat_files:
        parts = file.split(os.sep)
        date_folder, person_name = parts[-4], parts[-2]  # Extracting the person's name

        try:
            folder_date = pd.to_datetime(date_folder, format='%Y-%m-%d').date()
        except ValueError:
            continue

        expected_date_minus_one = folder_date - datetime.timedelta(days=1)
        key = f"{folder_date.strftime('%Y-%m-%d')}_{person_name}"

        if key not in dataframes:
            dataframes[key] = create_template_dataframe()
            start_column_index = 0
        else:
            if not dataframes[key].columns.empty:
                start_column_index = max(dataframes[key].columns) + 1
            else:
                start_column_index = 0

        parsed_data = parse_chat_file(file, expected_date_minus_one, person_name)
        dataframes[key], start_column_index = populate_dataframe(dataframes[key], parsed_data, start_column_index)

    return dataframes




def create_graphs(df, person_identifier, base_directory):
    graph_directory = os.path.join(base_directory, "Graphs")
    os.makedirs(graph_directory, exist_ok=True)

    # Sum the values for 'person' and 'other' messages for each minute
    person_chat_activity = df.iloc[:, 0::3].sum(axis=1)
    other_chat_activity = df.iloc[:, 1::3].sum(axis=1)
    
    # Find the first and last non-zero indices for chats
    non_zero_indices = person_chat_activity[person_chat_activity > 0].index
    first_chat_time = non_zero_indices[0] if not non_zero_indices.empty else df.index[0]
    last_chat_time = non_zero_indices[-1] if not non_zero_indices.empty else df.index[-1]

    # Creating the plot
    fig, ax = plt.subplots(figsize=(30, 10))
    fig.patch.set_facecolor('white')
    ax.set_facecolor('white')
    ax.xaxis.label.set_color('black')
    ax.yaxis.label.set_color('black')
    ax.title.set_color('black')
    ax.tick_params(axis='x', colors='black')
    ax.tick_params(axis='y', colors='black')


    # Plot the bar for 'person' messages
    ax.bar(df.index, person_chat_activity, color='lime', width=2, label='Counselor')

    # Plot the bar for 'other' messages, stacked on top of 'person' messages
    #ax.bar(df.index, other_chat_activity, color='darkgreen', width=1, label='Other Messages', bottom=person_chat_activity)
    ax.plot(df.index, other_chat_activity, color='darkgreen', linestyle=':', label='Student')
    # Draw white lines for the axes
    ax.axhline(0, color='black', linewidth=3)  # Changed color to black
    ax.axvline(first_chat_time, color='white', linewidth=3)
    #ax.axvline(df.index[0], color='black', linewidth=3)  # Changed color to black


    # Rotate x-axis labels to prevent overlap and increase label font sizes
    plt.xticks(rotation=90, fontsize=12)

    # Set y-axis to show every integer tick
    plt.yticks(np.arange(0, 11, 1), fontsize=12)

    # Set x-axis to show the range from the first chat to the last chat
    ax.set_xlim(first_chat_time, last_chat_time)
    #ax.set_xlim(df.index[0], df.index[-1])
    ax.xaxis.set_major_locator(ticker.MaxNLocator(96))  # Set locator for 15-minute intervals

    # Set y-axis dynamic range based on the maximum chat activity with a buffer
    max_activity = max(person_chat_activity.max(), other_chat_activity.max())
    ax.set_ylim(0, 11)

    # Increasing font size for labels and title
    ax.set_xlabel('Time', fontsize=12)
    ax.set_ylabel('Number of Chats', fontsize=12)
    ax.set_title(f'Chat Activity for {person_identifier}', fontsize=14)

    # Create and set the legend
    legend = ax.legend(facecolor='lightgray', edgecolor='black', fontsize=12, fancybox=True)
    for text in legend.get_texts():
        text.set_color('black')
        text.set_weight('bold')

    # Saving the graph
    graph_file_name = f"{person_identifier}.png"
    plt.savefig(os.path.join(graph_directory, graph_file_name), format='png', dpi=300, bbox_inches='tight')
    print(f"Graph saved as {graph_file_name}")

    plt.close(fig)


# Main script
date_directory = "C:\\Users\\maurice\\Documents\\Chat-Analyzer-V2\\Chat Folder from Drive\\2023-12-23-20231223T043422Z-001"
chat_files = list_chat_files(date_directory)
person_dataframes = process_person_chats(chat_files)

for person_identifier, df in person_dataframes.items():
    create_graphs(df, person_identifier, date_directory)

2023-12-23 11:10:09,465 - DEBUG - matplotlib data path: C:\Users\maurice\anaconda3\lib\site-packages\matplotlib\mpl-data
2023-12-23 11:10:09,477 - DEBUG - CONFIGDIR=C:\Users\maurice\.matplotlib
2023-12-23 11:10:09,479 - DEBUG - interactive is False
2023-12-23 11:10:09,479 - DEBUG - platform is win32
2023-12-23 11:10:09,479 - DEBUG - loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'nt', 'winreg', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'encodings.cp1252', 'site', 'os', 'stat', '_stat', 'ntpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'types', 'importlib', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib.util', 'importlib.abc', 'importlib.machinery', 'contextlib', 'collections', 'operator', '_operat

2023-12-23 11:10:09,595 - DEBUG - CACHEDIR=C:\Users\maurice\.matplotlib
2023-12-23 11:10:09,603 - DEBUG - Using fontManager instance from C:\Users\maurice\.matplotlib\fontlist-v330.json
2023-12-23 11:10:09,757 - WARNING - 
Bad key text.latex.preview in file C:\Users\maurice\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution
2023-12-23 11:10:09,757 - WARNING - 
Bad key mathtext.fallback_to_cm in file C:\Users\maurice\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib 